In [22]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [23]:
def edit_distance(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

In [68]:
def handle_residual(branch, length, preresidual):
    fot_op = np.array([float(p[2]) for p in branch]) * length
    idx_int = np.abs(fot_op-np.round(fot_op))<0.01
    idx_res = np.where(np.abs(fot_op-np.round(fot_op))>=0.01)[0].tolist()
    fot_op[idx_int] = np.round(fot_op[idx_int])
    total = np.sum(fot_op)
    residual = total - np.sum(np.floor(fot_op)) + preresidual
    if abs(residual - round(residual))<0.01:
        residual = round(residual)
    for i in range(len(branch)):
        branch[i][2] = int(np.floor(fot_op)[i])
    for i in idx_res:
        if residual >= 1:
            branch[i][2] += 1
            residual -= 1
    # print(branch)
    return residual

In [86]:
def gettree(fname):
    p = re.compile(r'requires a total of\s+(\w+)\.')
    fin = open(fname, 'r')
    l = fin.readline()
    while not l.rstrip() == '  -------      ---       ------':
        l = fin.readline()
        m = p.search(l)
        if m:
            nsize = int(m.groups()[0])
    branches = []
    while True:
        l = fin.readline()
        p = l.rstrip().split()
        if not len(p) == 3:
            break
        branches.append(p)
    fin.close()
    branches = sorted(branches, key=lambda a:int(a[0]))
    length = nsize / sum([float(p[2]) for p in branches])
    precurrent = ''
    branches_round = []
    tmp_branch = []
    residual = 0
    for i, p in enumerate(branches):
        current = p[0]
        if not current == precurrent:
            residual = handle_residual(tmp_branch, length, residual)
            branches_round.extend(tmp_branch)
            tmp_branch = [p]
            precurrent = current
        else:
            tmp_branch.append(p)
    residual = handle_residual(tmp_branch, length, residual)
    branches_round.extend(tmp_branch)
    # Handle multiple possible mutation case
    edges = []
    replace = {}
    k_ins = 0   
    for p in branches_round:
        editdistance = p[2]
        if editdistance == 0:
            replace[p[0]] = p[1]
        elif editdistance == 1:
            edges.append([p[0], p[1]])
        else:
            edges.append([p[0], 'ins'+str(k_ins)])
            k_ins += 1
            for k in range(editdistance-2):
                edges.append(['ins'+str(k_ins-1), 'ins'+str(k_ins)])
                k_ins += 1
            edges.append(['ins'+str(k_ins-1), p[1]])

    total = 0
    for i, edge in enumerate(edges):
        if edge[0] in replace:
            edge[0] = replace[edge[0]]
        if edge[1] in replace:
            edge[1] = replace[edge[1]]
    fin.close()
    return edges, nsize
def reorder(edges):
    current = 'G.L.'
    nodeid = 0
    name2id = {'G.L.':0}
    queue = [current]
    tree = []
    nodes = set()
    while len(queue) > 0:
        current = queue.pop(0)
        children = []
        for i in range(len(edges)):
            if current == edges[i][0] and edges[i][1] not in nodes:
                children.append(edges[i][1])
            if current == edges[i][1] and edges[i][0] not in nodes:
                children.append(edges[i][0])
        for i in range(len(children)):
            if children[i] not in name2id:
                nodeid += 1
                name2id[children[i]] = nodeid
        nodes.add(current)
        queue.extend(children)
        if len(children) > 0:
            tree.append([current] + children)
    return tree, name2id
def printtree(tree, name2id, size, fout=sys.stdout):
    # print(size, file=fout)
    for l in tree:
        print('\t'.join([str(name2id[name]) for name in l]), file=fout)

In [103]:
isim = 1
i = 25
fname = 'data/Simulated/sim%d/%05d.phy.outfile'%(isim, i)
fname = 'data/outfile'
edges, size = gettree(fname)
tree, name2id = reorder(edges)
fout = open('data/real.dnapars.tree','w')
printtree(tree, name2id, size, fout)
fout.close()
fout = open('data/real.dnapars.id','w')
print(','.join([str(name2id[k]) for k in name2id if k[0] in ['G','O']]), file=fout)
fout.close()

'0,38,40,46,53,65,66,67,78,81,84,85,88,93,94,95,97,100,101,103,104,106,111,116,117,118,119,128,134,139,141,147,148,149,153,158,159,164,166,167,168,171,174,175,184,187,188,193,196,197,203,206,207,210,213,214,218,221,230,234,235,236,248,249,256,262,265,266,267,270,280,282,283,284,285,289,293,294,299,300,301,302,303,304,305,306,307,308,309,310,311,312,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,331,332,333,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,358,362,364,366,368,369,370,372,378,381,382,386,387,392,394,395,397,398,399,405,408,409,410,412,413,414,415,416,417,418,419,426,427,429,431,432,433,435,436,437,438,439,440,442,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,465,466,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,488,495,496,497,498,499,500,502,503,505,506,507,508,509,510,517,521,522,524,525,526,527,529,530,534,537,540,542,543,544,545,546,547,548,549,551,552,555,558,562,563,565,566,567,568,569,5

In [98]:
name2id

{'11': 28,
 '3': 12,
 '5': 6,
 '8': 5,
 'G.L.': 0,
 'Obs10': 16,
 'Obs11': 8,
 'Obs12': 22,
 'Obs13': 11,
 'Obs14': 24,
 'Obs15': 29,
 'Obs16': 26,
 'Obs17': 35,
 'Obs18': 32,
 'Obs19': 21,
 'Obs2': 3,
 'Obs20': 10,
 'Obs3': 4,
 'Obs4': 2,
 'Obs5': 18,
 'Obs6': 27,
 'Obs7': 20,
 'Obs8': 17,
 'Obs9': 25,
 'ins0': 1,
 'ins1': 15,
 'ins10': 33,
 'ins11': 34,
 'ins2': 7,
 'ins3': 13,
 'ins4': 9,
 'ins5': 14,
 'ins6': 19,
 'ins7': 23,
 'ins8': 30,
 'ins9': 31}

In [ ]:
for isim in range(1, 10):
    for i in range(1, 501):
        fname = 'data/Simulated/sim%d/%05d.phy.outfile'%(isim, i)
        edges, size = gettree(fname)
        tree, name2id = reorder(edges)
        fout = open(fname.replace('outfile', 'out.edges'), 'w')
        printtree(tree, name2id, size, fout)
        fout.close()
        if not size == len(name2id)-1:
            print('sim%d_%05d:%d\t%d' % (isim, i, size, len(name2id)))
        

In [82]:
depth = {}
depth['G.L.'] = 0
for i, t in enumerate(tree):
    parent = t[0]
    children = t[1:]
    for c in children:
        depth[c] = depth[parent] + 1
depth

{'G.L.': 0,
 'ins56': 1,
 'ins55': 2,
 'ins54': 3,
 'ins53': 4,
 'ins52': 5,
 'ins51': 6,
 'ins50': 7,
 'ins49': 8,
 'ins48': 9,
 'ins47': 10,
 'ins46': 11,
 'ins45': 12,
 'ins44': 13,
 'ins43': 14,
 'ins42': 15,
 'ins41': 16,
 'ins40': 17,
 'ins39': 18,
 'ins38': 19,
 'ins37': 20,
 'ins36': 21,
 'ins35': 22,
 'ins34': 23,
 'ins33': 24,
 'ins32': 25,
 'ins31': 26,
 'ins30': 27,
 'ins29': 28,
 'ins28': 29,
 'ins27': 30,
 'ins26': 31,
 'ins25': 32,
 'ins24': 33,
 'ins23': 34,
 'ins22': 35,
 '1': 36,
 'ins0': 37,
 'Obs606': 37,
 'ins1': 37,
 'Obs308': 38,
 'ins410': 38,
 'ins413': 38,
 'ins414': 38,
 'ins2': 38,
 'ins411': 39,
 'Obs605': 39,
 '230': 39,
 'ins3': 39,
 'ins412': 40,
 'ins454': 40,
 'ins466': 40,
 'ins4': 40,
 'Obs562': 41,
 'ins455': 41,
 'ins467': 41,
 'ins5': 41,
 'ins416': 42,
 'ins456': 42,
 'ins468': 42,
 'ins6': 42,
 '209': 43,
 'ins457': 43,
 'ins469': 43,
 'ins7': 43,
 'Obs674': 44,
 'Obs683': 44,
 'Obs682': 44,
 'ins458': 44,
 'ins470': 44,
 'ins8': 44,
 'ins459': 

In [83]:
depth.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 37, 37, 38, 38, 38, 38, 38, 39, 39, 39, 39, 40, 40, 40, 40, 41, 41, 41, 41, 42, 42, 42, 42, 43, 43, 43, 43, 44, 44, 44, 44, 44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 47, 48, 48, 48, 49, 49, 49, 49, 50, 50, 50, 50, 51, 51, 51, 51, 51, 51, 52, 52, 52, 52, 52, 52, 53, 53, 53, 53, 53, 53, 53, 53, 54, 54, 54, 54, 54, 55, 55, 55, 55, 55, 55, 56, 56, 57, 57, 58, 58, 59, 59, 59, 59, 60, 60, 60, 60, 61, 61, 61, 61, 61, 61, 61, 61, 62, 62, 62, 62, 62, 62, 62, 62, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 69, 69, 6